In [1]:
data = '../data/2022_07_13_WTvsKO_allcompounds_neg.csv'

# import streamlit as st

In [2]:
import pandas as pd
import numpy as np

### fcn to upload data

In [3]:
def upload(file):
    df = pd.read_csv(file, header=2) #header=2 cuts off first two rows
    return df

In [4]:
df = upload(data)

### fcn to format data

In [5]:
def format(df):
    col = df.columns.tolist() #create a list of all column names
    main = [0,2,4,5] #Index of main columns we wish to keep and compare
    stop = col.index([col for col in df.columns if '.1' in col][0]) #index of duplicate columns we don't need
    intensities = col[16:stop] #intensity columns we wish to keep
    col_main = [] #Column names of kept columns
    for i in main:
        col_main.append(col[i])
    '''Create new filtered dataframe of important columns'''
    df_keep = df[col_main+intensities][:100]
    return df_keep 

In [6]:
df_keep = format(df)

### fcn to pick out pairs

In [7]:
def pick_pairs(df):
    '''Define lists and tolerances of each column to compare with itself'''
    mz = df['m/z']
    mz_tol = 1e-4
    rt = df['Retention time (min)']
    rt_tol = 1e-3
    ccs = df['CCS (angstrom^2)']
    ccs_tol = 1e-3

    '''Initialize a list for indexes to be held for each pair of matched values'''
    idxs = []
    '''Create nested for loop to compare i with j in each column'''
    for i in range(len(df)):
        for j in range(len(df)):
            '''Define checks for each column'''
            check_mz = np.isclose(mz[i], mz[j]+6.02, mz_tol)        
            check_rt = np.isclose(rt[i], rt[j], rt_tol)
            check_ccs = np.isclose(ccs[i], ccs[j], ccs_tol)
            
            '''Record results of each check (True/False)'''
            checks = [check_mz, check_rt, check_ccs]
            
            '''If all checks are true, append to list of pairs'''
            if all(checks) and i!=j:
                idxs.append([i,j])
                pairs = np.array(idxs)
            else:
                pass

    df_pairs = df.iloc[pairs.flatten()]
    return df_pairs

In [8]:
df_pairs = pick_pairs(df_keep)

### fcn to adjust mass of each pair

In [79]:
# df_adjusted = df_pairs
# for i in pairs:
#     df_adjusted['m/z'].iloc[i[0]]-=5
#     df_adjusted['m/z'].iloc[i[1]]-=11